In [1]:
using Pkg
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/JPEC/JPEC`


Plots.GRBackend()

In [2]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
equil_input = JPEC.Equilibrium.EquilInput(
    "beta_1.00",        # eq_filename
    "efit",          # eq_type
    "boozer",        # jac_type
    0.01,             # psilow
    1.0,             # psihigh
    100,             # mpsi (number of radial grid points)
    128              # mtheta (number of poloidal grid points)
)

# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_input)
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...
--- Julia Equilibrium Setup ---
Equilibrium file: beta_1.00
Type = efit, Jac_type = boozer
----------------------------------------
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separ

In [16]:
# 새 셀: JPEC 스플라인 직접 테스트
println("🔍 Direct JPEC Spline Testing")
println("="^40)

# 1D 프로파일 스플라인 테스트
println("\n📈 Testing 1D profile splines...")
try
    psi_test = [0.0, 0.25, 0.5, 0.75, 1.0]
    
    for psi in psi_test
        result = JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi)
        println("  ψ=$(psi): F=$(round(result[1], digits=3)), P=$(round(result[2], digits=3)), q=$(round(result[4], digits=3))")
    end
    
    println("  ✅ 1D spline evaluation successful")
    
catch e
    println("  ❌ 1D spline evaluation failed: $e")
end

# 2D 기하학적 스플라인 테스트
println("\n🗺️  Testing 2D geometry splines...")
try
    psi_test = 0.5
    theta_test = [0.0, 0.25, 0.5, 0.75]
    
    for theta in theta_test
        f, fx, fy = JPEC.SplinesMod.bicube_eval(plasma_eq.rzphi, psi_test, theta, 1)
        println("  (ψ=$(psi_test), θ=$(theta)): r²=$(round(f[1], digits=3)), shift=$(round(f[2], digits=3))")
    end
    
    println("  ✅ 2D spline evaluation successful")
    
catch e
    println("  ❌ 2D spline evaluation failed: $e")
end

🔍 Direct JPEC Spline Testing

📈 Testing 1D profile splines...
  ψ=0.0: F=3.472, P=0.035, q=10.118
  ψ=0.25: F=3.29, P=0.022, q=10.196
  ψ=0.5: F=3.135, P=0.012, q=10.833
  ψ=0.75: F=3.033, P=0.004, q=12.807
  ψ=1.0: F=3.0, P=0.0, q=21.972
  ✅ 1D spline evaluation successful

🗺️  Testing 2D geometry splines...
  (ψ=0.5, θ=0.0): r²=0.882, shift=0.0
  (ψ=0.5, θ=0.25): r²=1.985, shift=0.603
  (ψ=0.5, θ=0.5): r²=1.054, shift=1.228
  (ψ=0.5, θ=0.75): r²=1.985, shift=1.852
  ✅ 2D spline evaluation successful


In [17]:
# Fourfit v4 테스트 (v3 대신 v4 사용)
println("🚀 Testing Fourfit v4 with Real JPEC Equilibrium Data")
println("="^60)

# plasma_eq 존재 확인
if !@isdefined(plasma_eq)
    println("❌ plasma_eq is not defined - please run equilibrium setup first!")
    return
end

# 1. Fourfit v4 모듈 로드
include("/Users/seoda-eun/JPEC/fourfit_old_v4.jl")
using .FourfitOldV4

# 2. 실제 JPEC 평형으로 Fourfit 실행
println("\n📊 JPEC Equilibrium Info:")
println("  Type: $(typeof(plasma_eq))")
println("  Magnetic axis: R₀=$(round(plasma_eq.ro, digits=3)), Z₀=$(round(plasma_eq.zo, digits=6))")
println("  Flux normalization: ψ₀=$(round(plasma_eq.psio, digits=3))")
println("  Splines available:")
println("    - sq (1D profiles): $(plasma_eq.sq !== nothing ? "✅" : "❌")")
println("    - rzphi (2D geometry): $(plasma_eq.rzphi !== nothing ? "✅" : "❌")")
println("    - eq_quantities: $(plasma_eq.eq_quantities !== nothing ? "✅" : "❌")")

# 3. Fourfit 실행
println("\n🧪 Running Fourfit v4 with real JPEC data...")
global success, fourfit_data, eq_data = test_fourfit_with_jpec_equilibrium(plasma_eq)

if success
    println("\n🎉 Fourfit execution successful!")
    
    # 결과 분석
    println("\n📈 Results Analysis:")
    println("  Grid size: $(fourfit_data.mpsi+1) × $(fourfit_data.mtheta+1)")
    println("  Mode range: $(fourfit_data.mlow):$(fourfit_data.mhigh)")
    println("  Toroidal mode: n = $(fourfit_data.nn)")
    
    # 데이터 소스 확인
    println("\n🔍 Data Sources:")
    println("  1D Profiles: $(get(eq_data, "has_real_jpec", false) ? "Real JPEC splines" : "Mock data")")
    println("  2D Geometry: $(get(eq_data, "has_real_geometry", false) ? "Real JPEC bicubic" : "Simplified")")
    
    # 프로파일 정보
    println("\n📊 Equilibrium Profiles:")
    F_prof = eq_data["F_profile"]
    P_prof = eq_data["P_profile"] 
    q_prof = eq_data["q_profile"]
    
    println("  F profile range: [$(round(minimum(F_prof), digits=3)), $(round(maximum(F_prof), digits=3))] T⋅m")
    println("  P profile range: [$(round(minimum(P_prof), digits=3)), $(round(maximum(P_prof), digits=3))] T²")
    println("  q profile range: [$(round(minimum(q_prof), digits=3)), $(round(maximum(q_prof), digits=3))]")
    
    # MHD 행렬 정보
    if haskey(fourfit_data.cache, "mhd_matrix")
        matrix = fourfit_data.cache["mhd_matrix"]
        println("\n🔢 MHD Matrix Properties:")
        println("  Matrix size: $(size(matrix["A_matrix"]))")
        
        # 고유값 분석
        try
            eigenvals = eigvals(matrix["A_matrix"])
            real_eigs = real(eigenvals)
            println("  Eigenvalues:")
            println("    Range: [$(round(minimum(real_eigs), digits=3)), $(round(maximum(real_eigs), digits=3))]")
            println("    All positive: $(all(real_eigs .> 0) ? "✅" : "❌")")
            println("    Smallest eigenvalue: $(round(minimum(real_eigs), digits=6))")
            
            # 조건수
            cond_num = cond(matrix["A_matrix"])
            println("    Condition number: $(round(cond_num, digits=2))")
            
        catch e
            println("    Eigenvalue analysis failed: $e")
        end
    end
    
else
    println("\n❌ Fourfit execution failed!")
end

🚀 Testing Fourfit v4 with Real JPEC Equilibrium Data


  Activating project at `~/JPEC/JPEC`
┌ Warning: Assignment to `success` in soft scope is ambiguous because a global variable by the same name exists: `success` will be treated as a new local. Disambiguate by using `local success` to suppress this warning or `global success` to assign to the existing global variable.
└ @ nothing /Users/seoda-eun/JPEC/fourfit_old_v4.jl:1165
┌ Warning: Assignment to `fourfit_data` in soft scope is ambiguous because a global variable by the same name exists: `fourfit_data` will be treated as a new local. Disambiguate by using `local fourfit_data` to suppress this warning or `global fourfit_data` to assign to the existing global variable.
└ @ nothing /Users/seoda-eun/JPEC/fourfit_old_v4.jl:1165



📊 JPEC Equilibrium Info:
  Type: JPEC.Equilibrium.Types.PlasmaEquilibrium
  Magnetic axis: R₀=3.118, Z₀=-3.0e-6
  Flux normalization: ψ₀=1.367
  Splines available:
    - sq (1D profiles): ✅
    - rzphi (2D geometry): ✅
    - eq_quantities: ✅

🧪 Running Fourfit v4 with real JPEC data...
🧪 Testing fourfit with JPEC PlasmaEquilibrium
📋 Fourfit parameters:
  Grid: 17 × 25
  Perturbations: 5 modes
  Bandwidth: 2
  Mode range: -2:2
  Toroidal mode: n = 1

📊 Extracting equilibrium data...
📊 Extracting equilibrium data using JPEC spline system...
  ✓ JPEC PlasmaEquilibrium structure validated
  ✓ Magnetic axis: R₀=3.1176357129137466, Z₀=-2.9184696943092166e-6
  ✓ Flux normalization: ψ₀=1.367210836
  📈 Evaluating 1D profiles...
    🎯 Using real JPEC spline data...
    ✅ Real JPEC 1D profiles extracted successfully
      F range: [3.0, 3.472]
      P range: [0.0, 0.035]
      q range: [10.118, 21.972]
  🗺️  Evaluating 2D geometry...
    🎯 Using real JPEC geometric data...
    ✅ Real JPEC 2D geo

┌ Warning: Assignment to `eq_data` in soft scope is ambiguous because a global variable by the same name exists: `eq_data` will be treated as a new local. Disambiguate by using `local eq_data` to suppress this warning or `global eq_data` to assign to the existing global variable.
└ @ nothing /Users/seoda-eun/JPEC/fourfit_old_v4.jl:1165


ErrorException: type OldFourfitData has no field cache

In [9]:
# The plasma_eq object contains the final 1D profile spline, `sq`.
# We will evaluate it on a fine grid to get smooth plot lines.
psi_norm_grid = range(0.0, 1.0, length=200)

# spline_eval returns the function values (and derivatives if requested).
# For sq, the quantities are: 1:F, 2:P*mu0, 3:Toroidal Flux, 4:q
f_profiles = JPEC.spline_eval(plasma_eq.sq, collect(psi_norm_grid))

# Extract each profile into its own variable for clarity
F_profile      = f_profiles[:, 1]
P_profile      = f_profiles[:, 2]
tor_flux_prof  = f_profiles[:, 3]
q_profile      = f_profiles[:, 4]

UndefVarError: UndefVarError: `spline_eval` not defined in `JPEC`
Suggestion: check for spelling errors or missing imports.

In [8]:

p = plot(layout=(2, 2), size=(900, 700), margin=5Plots.mm)

# Plot 1: F = R*B_phi
plot!(p[1], psi_norm_grid, F_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="F [T⋅m]",
    title="Toroidal Field Function (F)",
    legend=false,
    lw=2
)

# Plot 2: Pressure
plot!(p[2], psi_norm_grid, P_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="μ₀P [T²]",
    title="Pressure Profile",
    legend=false,
    lw=2
)

# Plot 3: Safety Factor (q)
plot!(p[3], psi_norm_grid, q_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="q",
    title="Safety Factor (q) Profile",
    legend=false,
    lw=2
)

# Plot 4: Toroidal Flux
plot!(p[4], psi_norm_grid, tor_flux_prof,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="Φ / (2π)² [Wb]",
    title="Toroidal Flux Profile",
    legend=false,
    lw=2
)

# Display the final plot
p

UndefVarError: UndefVarError: `F_profile` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
# block 6
# --- Plot Straight-Field-Line Angle (θ) Contours ---

p_theta = contour(
    R_grid, 
    Z_grid, 
    THETA_values_on_grid,
    title="Straight-Field-Line Angle (θ) Contours",
    xlabel="R [m]",
    ylabel="Z [m]",
    levels=20,
    c=:hsv,
    colorbar_title="Normalized θ",
    aspect_ratio=:equal,
    linewidth=1.5
)

# Mark the magnetic axis
scatter!(p_theta, [plasma_eq.ro], [plasma_eq.zo], label="Magnetic Axis", mc=:black, ms=5)

display(p_theta)

UndefVarError: UndefVarError: `R_grid` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
# 노트북 셀 1: JPEC 로드
using Pkg
Pkg.activate("/Users/seoda-eun/JPEC/JPEC")
using JPEC

# 노트북 셀 2: Fourfit 포함
include("/Users/seoda-eun/JPEC/fourfit_old_v4.jl")
using .FourfitOldV4

# 노트북 셀 3: 실제 JPEC 평형 사용
# plasma_eq는 실제 JPEC 평형 객체라고 가정
success, fourfit_data, eq_data = test_fourfit_with_jpec_equilibrium(plasma_eq)

  Activating project at `~/JPEC/JPEC`
  Activating project at `~/JPEC/JPEC`


🧪 Testing fourfit with JPEC PlasmaEquilibrium
📋 Fourfit parameters:
  Grid: 129 × 25
  Perturbations: 33 modes
  Bandwidth: 2
  Mode range: -12:20
  Toroidal mode: n = 1

📊 Extracting equilibrium data...
📊 Extracting equilibrium data using JPEC spline system...
  ✓ JPEC PlasmaEquilibrium structure validated
  ✓ Magnetic axis: R₀=3.1176357129137466, Z₀=-2.9184696943092166e-6
  ✓ Flux normalization: ψ₀=1.367210836
  📈 Evaluating 1D profiles...
    🎯 Using real JPEC spline data...
    ✅ Real JPEC 1D profiles extracted successfully
      F range: [3.0, 3.472]
      P range: [0.0, 0.035]
      q range: [10.118, 21.972]
  🗺️  Evaluating 2D geometry...
    🎯 Using real JPEC geometric data...
    ✅ Real JPEC 2D geometry extracted successfully
  🧮 Computing profile derivatives...
✅ JPEC equilibrium data extraction complete
  🎯 Status: Full real JPEC data used

📐 Extracting geometric data...
📐 Extracting geometric data from JPEC...
  Using real JPEC geometric data...
  ✅ Real JPEC geometric data

(true, OldFourfitData(128, 24, 2, 33, -12, 20, 1, Main.FourfitOldV4.SimpleSplineType{Float64}([0.0, 0.0078125, 0.015625, 0.0234375, 0.03125, 0.0390625, 0.046875, 0.0546875, 0.0625, 0.0703125  …  0.9296875, 0.9375, 0.9453125, 0.953125, 0.9609375, 0.96875, 0.9765625, 0.984375, 0.9921875, 1.0], [0.0, 0.2617993877991494, 0.5235987755982988, 0.7853981633974483, 1.0471975511965976, 1.3089969389957472, 1.5707963267948966, 1.8325957145940461, 2.0943951023931953, 2.356194490192345  …  3.9269908169872414, 4.1887902047863905, 4.4505895925855405, 4.71238898038469, 4.974188368183839, 5.235987755982989, 5.497787143782138, 5.759586531581287, 6.021385919380437, 6.283185307179586], [NaN Inf … -3.046934458274326e11 Inf; 3.158963253585444 350.981437293284 … 4.658378029806704e9 3.158963253585444; … ; 0.022266547432566654 0.15819397319403583 … 3041.233115511038 0.022266547432566654; 0.022143628723606278 350.28752430237165 … 399605.35548610357 0.022143628723606278;;; 0.0 0.0 … -1.796093879841295e10 0.0; 7.7